In [1]:
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
import re
import string
import json
import pickle
from collections import defaultdict

pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

from bs4 import BeautifulSoup
# The "requests" library makes working with HTTP requests easier
# than the built-in urllib libraries.
import requests
from urllib2 import urlopen

In [9]:
test1 = open("anime_pages.p",'rb') 
anime = pickle.load(test1)  
test1.close()

In [11]:
root = 'http://www.anime-planet.com'

all_anime = []
for page in anime:
    soup = BeautifulSoup(page.text, "html.parser")
    animelist = soup.find('ul',attrs={"class": "cardDeck pure-g cd-narrow"}).find_all("li")
    for ani in animelist:
        all_anime.append(root + ani.find('a').get('href'))

In [112]:
%%time
anime_reviews1 = defaultdict(list)
for i in xrange(len(all_anime)):
    review = requests.get(all_anime[i]+"/reviews")
    soup1 = BeautifulSoup(review.text, "html.parser")
    pages = soup1.find('section',attrs={'pure-g'}).find('div',attrs='pure-u-1 pure-u-md-4-5')
    if pages is not None:
        pages = pages.find('div',attrs='pagination aligncenter')
        if pages is not None:
            all_li = pages.ul.find_all('li')
            maxpage = int(all_li[len(all_li)-2].a.contents[0])
        else:
            maxpage=1
    else:
        maxpage=1

    for j in xrange(1,maxpage+1):
        anime_reviews1[all_anime[i].split('/')[-1]].append(requests.get(all_anime[i]+"/reviews?page="+str(j)))
        time.sleep(1)

CPU times: user 3min 32s, sys: 21.2 s, total: 3min 54s
Wall time: 2h 44min 30s


In [2]:
#pickle.dump(anime_reviews, open("anime_reviews.p", "wb"))

test3 = open("anime_reviews.p",'rb') 
anime_reviews = pickle.load(test3)  
test3.close()

In [ ]:
#anime_reviews['digimon-adventure-tri-movie-2'][0].text

In [127]:
reviewsdict = {}
for anime in anime_reviews.keys():
    animedict = {anime:{}}
    for i in range(len(anime_reviews[anime])):
        soup = BeautifulSoup(anime_reviews[anime][i].text, "html.parser")
        body = soup.find('div',attrs={"class": "pure-u-1 pure-u-md-4-5"})
        if body is not None:
            sections = body.find_all('section', attrs={'class':'pure-g'})
            names = [i.find('a',attrs={'itemprop':'author'}).contents[0] for i in sections if i.find('a',attrs={'itemprop':'author'}) is not None]
            scores = [i.find_all('div',attrs={'class':'pure-u-1-5'}) for i in sections if i.find_all('div',attrs={'class':'pure-u-1-5'}) != []]
            story = [i[0].contents[0] for i in scores]
            animation = [i[1].contents[0] for i in scores]
            sound = [i[2].contents[0] for i in scores]
            characters = [i[3].contents[0] for i in scores]
            overall = [i[4].contents[0] for i in scores]
            reviews = [i.contents[1].string for i in body.find_all('section', attrs={'class':'userContent readMore'})]
            temp = dict(zip(names,zip(story,animation,sound,characters,overall,reviews)))
            animedict[anime].update(temp)
    reviewsdict.update(animedict)


In [2]:
# with open('all_reviews.json', 'w') as fp:
#       json.dump(reviewsdict, fp)

with open('all_reviews.json','r') as data_file:    
    reviewsdict = json.load(data_file)

In [3]:
len(reviewsdict)

7282

In [8]:
reviewlist = []
for i in reviewsdict.keys():
    reviewlist+=reviewsdict[i].keys()
users = list(set(reviewlist))

In [5]:
len(users),len(reviewlist)

(4420, 17019)

#### Getting User Profile

In [9]:
users[0]

u'stangace20'

In [15]:
%%time
root = 'http://www.anime-planet.com/users/'
user_profiles = {}
for i in xrange(len(users)):
    user = requests.get(root+users[i])
    soup = BeautifulSoup(user.text, "html.parser")
    user_profiles[users[i]]=soup
    time.sleep(1)

CPU times: user 3min 6s, sys: 17.1 s, total: 3min 23s
Wall time: 33min 42s


In [ ]:
pickle.dump(user_profiles, open("user_profiles.p", "wb"))

test5 = open("user_profiles.p",'rb') 
user_profiles = pickle.load(test5)  
test5.close()

In [22]:
for i in user_profiles.keys():
    print user_profiles[i].find('div',attrs={'class':'pure-u-3-5 pure-u-md-4-5'})
    break

<div class="pure-u-3-5 pure-u-md-4-5">
<a class="profile-avatar">
<img alt="stangace20's avatar" id="user-avatar" src="/images/users/avatars/43848.jpg?timestamp=1312502182"/></a>
<h2 class="tooltip" data-class="lastOnline-tooltip hoverable" id="profileName" title="Last online May 27, 2014">
<a class="lastOnline away"></a>
<a href="/users/stangace20">stangace20</a>
</h2>
<ul class="userStats">
<li><i class="fa fa-home"></i>San Diego,CA</li>
<li><i class="fa fa-calendar"></i>Joined Aug 27, 2008</li>
<li>
<i class="fa fa-user"></i>
45 </li>
</ul>
</div>
